# DSCB310 – Abgabe Übungsblatt 1 (WS22/23)


Matrikelnummern:  80001, 79679 

# Gliederung
### 1. Data Quality
### 2. Given Exercise
### 3. Department Manager 
### 4. Head of Department

# Data Quality

In [32]:
#Imports 
import pandas as pd
import numpy as np
import datetime

# Data Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split


In [33]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Column temp: decimal point shifted 2 digits

In [34]:
df = pd.read_parquet("escooter_history_2022.parquet")
#dividing outliers by 100
df.loc[df.temp >45, "temp"] = df.temp.div(100)

![alt text](s1.png "askldnlasnd")

# Column holiday and workingday: few values on 41 days that are incosistent with its day value

In [35]:
# getting specific values
df_33 = df.groupby(pd.Grouper(key="datetime",freq="D")).agg({"holiday":"mean","workingday":"mean"}).reset_index()
df_round = df_33.loc[(~df_33.workingday.isin([1,0]) | ~df_33.holiday.isin([0,1]))].round().reset_index(drop=True)

# comparing and alligning for every 1 or 0 value of holiday and workingday
df_holiday_1 = df_round.loc[df_round.holiday == 1]
df_holiday_1 = df_holiday_1.datetime.dt.date.to_list()
df.loc[df.datetime.dt.date.isin(df_holiday_1),"holiday"] = 1
df_holiday_0 = df_round.loc[df_round.holiday == 0]
df_holiday_0 = df_holiday_0.datetime.dt.date.to_list()
df.loc[df.datetime.dt.date.isin(df_holiday_0),"holiday"] = 0
df_workingday_1 = df_round.loc[df_round.workingday == 1]
df_workingday_1 = df_workingday_1.datetime.dt.date.to_list()
df.loc[df.datetime.dt.date.isin(df_workingday_1),"workingday"] = 1
df_workingday_0 = df_round.loc[df_round.workingday == 0]
df_workingday_0 = df_workingday_0.datetime.dt.date.to_list()
df.loc[df.datetime.dt.date.isin(df_workingday_0),"workingday"] = 0



# Column Humidity: 0 and 100 values that make no sense

![alt text](s3.png "askldnlasnd")

In [36]:
# only taking values between 0.1% and 99.9% 
df = df.loc[df.humidity.between(df.humidity.quantile(0.001), df.humidity.quantile(0.999))]

We drop all rows with values 100 and 0

In [37]:
#second option
#df.drop(df.loc[(df.weather == "clear, few clouds") & (df.humidity == 100)].index, inplace= True)
#df.drop(df.loc[(df.weather == "light snow or rain or thunderstorm") & (df.humidity == 0)].index, inplace= True)

# Column Humidity: only 30 unique values on 3,7 million rows


-> possibly binning 

In [38]:
df.windspeed.describe().round()

count    3754232.0
mean          13.0
std            8.0
min            0.0
25%            7.0
50%           13.0
75%           19.0
max           57.0
Name: windspeed, dtype: float64

In [39]:
df.windspeed.nunique()

30

# Given Erxercise

# Exercise 1
## Which weather attributes influence total unregistered customer the most

In [40]:
#only on unregistered customer
df_aufgabe1 = df[df["registered_customer"] == False].copy()
df_aufgabe1 = pd.get_dummies(df_aufgabe1)
df_slice3 = df_aufgabe1.groupby(pd.Grouper(key="datetime",freq="min")).agg(
 Total_customer=("registered_customer","count"),
 atemp=("atemp","mean"),
 temp=("temp","mean"),
 humidity=("humidity","mean"),
 windspeed=("windspeed","mean"),
 clear_few_clouds=("weather_clear, few clouds","mean"),
 cloudy_mist=("weather_cloudy, mist","mean"),
 heacy_rain_thunderstorm_snow_icepallets=("weather_heacy rain or thunderstorm or snow or ice pallets","mean"),
 ligth_snow_rain_thunderstorm=("weather_light snow or rain or thunderstorm","mean")).reset_index()
df_slice3.dropna(inplace=True)

In [41]:
# correlation of every weather attribute
fig= px.imshow(df_slice3[['Total_customer','temp', 'atemp',
       'humidity', 'windspeed']].corr(),text_auto=True,aspect="auto", color_continuous_scale= px.colors.sequential.Brwnyl)
fig.update_layout(title= dict(text= "<b>Exercise 1 Correlation</b>", xanchor= "center", yanchor= "top", y=0.98, x=0.46))

### -> temp 0.196 and atemp 0.193

### -> temp correlates slightly better than atemp

In [42]:

# X = df_slice3[['temp', 'atemp',
#        'humidity', 'windspeed',"clear_few_clouds","cloudy_mist","heacy_rain_thunderstorm_snow_icepallets","ligth_snow_rain_thunderstorm"]]
X = df_slice3[['temp', 'atemp',
       'humidity', 'windspeed']]
Y = df_slice3['Total_customer'] 


scaler = StandardScaler() 
Xs = pd.DataFrame(scaler.fit_transform(X[X.columns]))
Xs.columns = X.columns
regr = linear_model.LinearRegression()
regr.fit(Xs, Y)
pred = regr.predict(Xs)
print(Xs.columns.values)
print('Coefficients: \n', regr.coef_)
# print('Intercept: \n', regr.intercept_)
# print('On Training set:')
# print("Mean squared error: %.2f" % mean_squared_error(Y, pred))
# print("Variance of target variable: %.2f" % np.var(Y))
print('Variance explained: %.2f' % r2_score(Y, pred))

['temp' 'atemp' 'humidity' 'windspeed']
Coefficients: 
 [ 0.03972404  0.20977719 -0.22781188  0.0070811 ]
Variance explained: 0.06


### -> atemp influences the total customers more than temp,

### -> although relative low r2 score

# Exercise 2

# Which weather attributes influence total unregistered customer the most on a holiday with windspeed greater than 35

In [43]:
# has to be a holiday and windspeed > 35
df_aufgabe2 = df_aufgabe1[(df_aufgabe1.holiday==1)&(df_aufgabe1.windspeed>35)]
df_slice4 = df_aufgabe2.groupby(pd.Grouper(key="datetime",freq="min")).agg(
 Total_customers=("registered_customer","count"),
 holiday=("holiday","mean"),
 workingday=("workingday","mean"),
 temp=("temp","mean"),
 atemp=("atemp","mean"),
 humidity=("humidity","mean"),
 windspeed=("windspeed","mean")).reset_index()
df_slice4.dropna(inplace=True)

In [44]:
# correlation of weather attributes
fig = px.imshow(df_slice4[['Total_customers','temp', 'atemp',
       'humidity', 'windspeed']].corr(),text_auto=True,aspect="auto", color_continuous_scale= px.colors.sequential.Brwnyl)
fig.update_layout(title= dict(text= "<b>Exercise 2 Correlation</b>", xanchor= "center", yanchor= "top", y=0.98, x=0.46))

### -> temp -0.07 and atemp -0.03

### -> temp correlates slightly better than atemp, although both are really low

In [45]:
# X = df_slice4[['temp', 'atemp',
#        'humidity', 'windspeed',"clear_few_clouds","cloudy_mist","heacy_rain_thunderstorm_snow_icepallets","ligth_snow_rain_thunderstorm"]]
X = df_slice4[['temp', 'atemp',
       'humidity', 'windspeed']]
Y = df_slice4['Total_customers'] 


scaler = StandardScaler() 
Xs = pd.DataFrame(scaler.fit_transform(X[X.columns]))
Xs.columns = X.columns
regr = linear_model.LinearRegression()
regr.fit(Xs, Y)
pred = regr.predict(Xs)
print(Xs.columns.values)
print('Coefficients: \n', regr.coef_)
# print('Intercept: \n', regr.intercept_)
# print('On Training set:')
# print("Mean squared error: %.2f" % mean_squared_error(Y, pred))
# print("Variance of target variable: %.2f" % np.var(Y))
print('Variance explained: %.2f' % r2_score(Y, pred))

['temp' 'atemp' 'humidity' 'windspeed']
Coefficients: 
 [-1.83833963  1.19194204  0.80104009 -0.12735124]
Variance explained: 0.03


### -> atemp influences total customers more than temp 

### -> because our r2 score is even lower than in the first test and both temp und atemp take either negative or positive influence on our number of customers, we start to wonder if temp or atemo are even relevant for this case 

# Department Manager

### the trend of our total customers between 2020 and the beginning of 2022

In [46]:
df_copy = df.copy()
df_w = df_copy.groupby(["registered_customer", pd.Grouper(key="datetime", freq= "W")]).agg(
               humidity= ("humidity", "mean"), 
               windspeed= ("windspeed", "mean"), 
               temp= ("temp", "mean"), 
               atemp= ("atemp", "mean"), 
               reg_customer= ("registered_customer", "count"), 
               holiday= ("holiday", "sum"), 
               workingday= ("workingday", "sum"),  
                ).reset_index()

In [47]:
# Customers-Scatter
fig = px.scatter(df_w, x= df_w.datetime, 
              y= df_w.reg_customer, 
              color= df_w.registered_customer, 
              labels= dict(reg_customer="<b>Total Customers</b>", datetime="<b>Datetime</b>", registered_customer="<b>Registered Customers</b>"), 
              marginal_y = "box", 
              trendline= "rolling", 
              trendline_options= dict(window= 5, win_type="gaussian", function_args=dict(std=2)),
              trendline_color_override= px.colors.sequential.Brwnyl[1],
              color_discrete_sequence= [px.colors.sequential.Brwnyl[3],px.colors.sequential.Brwnyl[5]]
              )

# Decline1
fig.add_vrect(x0="2020-7-26", 
              x1="2021-01-26", 
              col=1,
              annotation_text= "decline1 (W 20/21)", 
              annotation_position="top left",
              fillcolor=px.colors.sequential.Brwnyl[2], 
              opacity=0.2, 
              line_width=0,
              annotation_font_color = px.colors.sequential.Brwnyl[6],
              annotation_font_size = 13) 

# Decline2 
fig.add_vrect(x0="2021-09-2", 
              x1="2022-01-2", 
              col=1,
              annotation_text= "decline2 (W 21/22)", 
              annotation_position="top left",
              fillcolor=px.colors.sequential.Brwnyl[2], 
              opacity=0.2, 
              line_width=0,
              annotation_font_color = px.colors.sequential.Brwnyl[6],
              annotation_font_size = 13)
             
# Reg-Customer-True-Mean 
fig.add_hline(y= df_w.loc[df_w.registered_customer == True].reg_customer.mean(), 
              line_dash= "dot", 
              annotation_text= "Reg_Cust_True",
              annotation_position="bottom left",
              col= 1,
              opacity = 0.5,
              line_color = px.colors.sequential.Brwnyl[5],
              annotation_font_color = px.colors.sequential.Brwnyl[5],
              annotation_font_size= 12)

# Reg-Customer-False-Mean
fig.add_hline(y= df_w.loc[df_w.registered_customer == False].reg_customer.mean(), 
              line_dash= "dot", 
              annotation_text= "Reg_Cust_False",
              annotation_position="bottom left",
              col= 1,
              opacity = 0.5,
              line_color = px.colors.sequential.Brwnyl[4],
              annotation_font_color = px.colors.sequential.Brwnyl[4],
              annotation_font_size= 12)


fig.add_annotation(
              x= "2020-01-01",
              y= df_w.loc[df_w.registered_customer == False].reg_customer.mean(),
              arrowhead= 1,
              showarrow= True,
              arrowcolor= px.colors.sequential.Brwnyl[4],
              font= dict(size= 12, color= px.colors.sequential.Brwnyl[4]),
              text= f"Median: {df_w.loc[df_w.registered_customer == False].reg_customer.mean().round(2)}")

fig.add_annotation(
              x= "2020-01-01",
              y= df_w.loc[df_w.registered_customer == True].reg_customer.mean(),
              arrowhead= 1,
              showarrow= True,
              arrowcolor= px.colors.sequential.Brwnyl[5],
              font= dict(size= 12, color= px.colors.sequential.Brwnyl[5]),
              text= f"Median: {df_w.loc[df_w.registered_customer == True].reg_customer.mean().round(2)}")
         

fig.update_annotations(opacity= 0.4)
fig.update_traces(showlegend=True)
fig.update_xaxes(title_text= "<b>Box</b>",row=1, col=2)
fig.update_layout(title= dict(text= "<b>Quantity Comparison of Weekly Registered and Unregistered Customers</b>", xanchor= "center", yanchor= "top", y=0.98, x=0.46),
                  height= 600, legend=dict(groupclick="toggleitem"), font= dict(size= 14))

### -> overall positive trendline over the last 2 years

## Total Customer Compairson 2020 / 2021

In [48]:
df1 = df.copy()

In [49]:
df_bar_cust_type= df1.groupby(["registered_customer", pd.Grouper(key= "datetime", freq= "Y")]).agg(total_customers = ("registered_customer", "count")).reset_index()

In [50]:
df_bar_2020= df_bar_cust_type.loc[df_bar_cust_type.datetime.dt.year.isin([2020])].reset_index(drop=True)
df_bar_2021= df_bar_cust_type.loc[df_bar_cust_type.datetime.dt.year.isin([2021])].reset_index(drop= True)

In [51]:
df_bar_2020

,registered_customer,datetime,total_customers
0,False,2020-12-31,271262
1,True,2020-12-31,1127191


In [52]:
fig_bar = make_subplots(rows= 1, cols= 2, shared_xaxes= True, shared_yaxes= True, horizontal_spacing= 0.01)

fig_bar.add_trace(
 go.Bar(
  x= df_bar_2020.loc[df_bar_2020.registered_customer == False].registered_customer,
  y= df_bar_2020.loc[df_bar_2020.registered_customer == False].total_customers,
  marker= dict(color= px.colors.sequential.Brwnyl[3]),
  name= "Unregistered 2020",
  legendgroup= "1",
  legendgrouptitle_text= "Unregistered:"
 ),
 row= 1,
 col= 1
)

fig_bar.add_trace(
 go.Bar(
  x= df_bar_2020.loc[df_bar_2020.registered_customer == True].registered_customer,
  y= df_bar_2020.loc[df_bar_2020.registered_customer == True].total_customers,
  marker= dict(color= px.colors.sequential.Brwnyl[5]),
  name= "Registered 2020",
  legendgroup= "2",
  legendgrouptitle_text= "Registered:"
 ),
 row= 1,
 col= 1
)

fig_bar.add_trace(
 go.Bar(
  x= df_bar_2021.loc[df_bar_2021.registered_customer == False].registered_customer,
  y= df_bar_2021.loc[df_bar_2021.registered_customer == False].total_customers,
  marker= dict(color= px.colors.sequential.Brwnyl[3]),
  name= "Unregistered 2021",
  legendgroup= "1",
  insidetextanchor= "middle",
  textfont= dict(color="white"),
  text= f"+{((df_bar_2020[df_bar_2020.registered_customer ==False].total_customers / df_bar_2021[df_bar_2021.registered_customer ==False].total_customers)*100).round(2)[0]}%"
 ),
 row= 1,
 col= 2
)

fig_bar.add_trace(
 go.Bar(
  x= df_bar_2021.loc[df_bar_2021.registered_customer == True].registered_customer,
  y= df_bar_2021.loc[df_bar_2021.registered_customer == True].total_customers,
  marker= dict(color= px.colors.sequential.Brwnyl[5]),
  name= "Registered 2021",
  legendgroup= "2",
  insidetextanchor= "middle",
  textfont= dict(color="white"),
  text= f"+{((df_bar_2020[df_bar_2020.registered_customer ==True].total_customers / df_bar_2021[df_bar_2021.registered_customer ==True].total_customers)*100).round(2)[1]}%"
 ),
 row= 1,
 col= 2
)

fig_bar.add_annotation(
 x= "false",
 y= df_bar_2020[df_bar_2020.registered_customer == False].total_customers.max(),
 arrowhead= 1,
 showarrow= True,
 arrowcolor= px.colors.sequential.Brwnyl[3],
 font= dict(size= 12, color= px.colors.sequential.Brwnyl[3]),
 text= f"{df_bar_2020[df_bar_2020.registered_customer == False].total_customers[0]} Customers",
 row= 1, col=1
)

fig_bar.add_annotation(
 x= "true",
 y= df_bar_2020[df_bar_2020.registered_customer == True].total_customers.max(),
 arrowhead= 1,
 showarrow= True,
 arrowcolor= px.colors.sequential.Brwnyl[5],
 font= dict(size= 12, color= px.colors.sequential.Brwnyl[5]),
 text= f"{df_bar_2020[df_bar_2020.registered_customer == True].total_customers[1]} Customers",
 row=1, col=1
)

fig_bar.add_annotation(
 x= "false",
 y= df_bar_2021[df_bar_2021.registered_customer == False].total_customers.max(),
 arrowhead= 1,
 showarrow= True,
 arrowcolor= px.colors.sequential.Brwnyl[3],
 font= dict(size= 12, color= px.colors.sequential.Brwnyl[3]),
 text= f"{df_bar_2021[df_bar_2021.registered_customer == False].total_customers[0]} Customers",
 col= 2, row=1
)

fig_bar.add_annotation(
 x= "true",
 y= df_bar_2021[df_bar_2021.registered_customer == True].total_customers.max(),
 arrowhead= 1,
 showarrow= True,
 arrowcolor= px.colors.sequential.Brwnyl[5],
 font= dict(size= 12, color= px.colors.sequential.Brwnyl[5]),
 text= f"{df_bar_2021[df_bar_2021.registered_customer == True].total_customers[1]} Customers",
 col=2,row= 1
)

fig_bar.update_layout(legend=dict(groupclick="toggleitem"),title= dict(text= "<b>Total Customer Comparison 2020/2021</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.9, x=0.43), legend_title_text = "<b>Registered Customers</b>",)
fig_bar.update_yaxes(title_text="<b>Total Customers</b>", secondary_y=False, col=1)
fig_bar.update_xaxes(title_text="<b>2020</b>", col= 1)
fig_bar.update_xaxes(title_text="<b>2021</b>", col= 2)
fig_bar.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig_bar.show()


### compared to our total customers in 2020 we got an increase on total customers of 63.6%(on unregistered customer) and 58.9%(on registered customer)

# Decline of total customers in winter

In [53]:
df_cust_temp = df.copy()
df_cust_temp = pd.get_dummies(df_cust_temp, prefix=["w"])

In [54]:
df_w = df_cust_temp.groupby(["registered_customer", pd.Grouper(key="datetime", freq= "W")]).agg(
               humidity= ("humidity", "mean"), 
               windspeed= ("windspeed", "mean"), 
               temp= ("temp", "mean"), 
               atemp= ("atemp", "mean"), 
               reg_customer= ("registered_customer", "count"), 
               holiday= ("holiday", "sum"), 
               workingday= ("workingday", "sum"),  
               clear_few_clouds = ("w_clear, few clouds", "sum"),
               w_cloudy_mist = ("w_cloudy, mist", "sum"),
               w_heacy_rain_thund_snow_icepal= ("w_heacy rain or thunderstorm or snow or ice pallets", "sum"), 
               w_light_snow_rain_thund= ("w_light snow or rain or thunderstorm", "sum")).reset_index()
df_w.head()

,registered_customer,datetime,humidity,windspeed,temp,atemp,reg_customer,holiday,workingday,clear_few_clouds,w_cloudy_mist,w_heacy_rain_thund_snow_icepal,w_light_snow_rain_thund
0,False,2020-01-05,75.681818,16.941545,16.165714,19.566180,462,0.0,0.0,122.0,293.0,0,47.0
1,False,2020-01-12,42.836826,15.772793,9.044551,10.791407,668,0.0,546.0,547.0,121.0,0,0.0
2,False,2020-01-19,44.462908,15.305623,9.813234,11.712122,674,0.0,201.0,475.0,196.0,0,3.0
3,False,2020-01-26,46.770459,14.862526,8.011776,9.945649,501,117.0,237.0,259.0,229.0,0,13.0
4,False,2020-02-02,67.263435,7.021275,8.674373,11.812150,949,0.0,342.0,591.0,319.0,0,39.0


In [55]:
df_wth = df_cust_temp.groupby(pd.Grouper(key="datetime", freq= "W")).agg(
               humidity= ("humidity", "mean"), 
               windspeed= ("windspeed", "mean"), 
               temp= ("temp", "mean"), 
               atemp= ("atemp", "mean"), 
               reg_customer= ("registered_customer", "count"), 
               holiday= ("holiday", "sum"), 
               workingday= ("workingday", "sum"),  
               clear_few_clouds = ("w_clear, few clouds", "sum"),
               w_cloudy_mist = ("w_cloudy, mist", "sum"),
               w_heacy_rain_thund_snow_icepal= ("w_heacy rain or thunderstorm or snow or ice pallets", "sum"), 
               w_light_snow_rain_thund= ("w_light snow or rain or thunderstorm", "sum")).reset_index()
df_wth.head()

,datetime,humidity,windspeed,temp,atemp,reg_customer,holiday,workingday,clear_few_clouds,w_cloudy_mist,w_heacy_rain_thund_snow_icepal,w_light_snow_rain_thund
0,2020-01-05,74.055431,16.068791,15.559339,18.805605,1786,0.0,0.0,470.0,1025.0,0,291.0
1,2020-01-12,45.588223,14.179146,8.481788,10.429790,9408,0.0,7627.0,7937.0,1423.0,0,48.0
2,2020-01-19,50.054626,14.580899,8.061622,9.903504,9025,0.0,6573.0,6373.0,2406.0,0,246.0
3,2020-01-26,52.770041,15.695682,8.580082,10.427223,7784,1000.0,5103.0,4302.0,3325.0,0,157.0
4,2020-02-02,67.422727,8.282825,8.351629,11.194581,10340,0.0,6317.0,5781.0,3602.0,25,932.0


In [56]:
fig3 = make_subplots(cols= 2, rows= 1, horizontal_spacing= 0.12, specs= [[{"secondary_y": True}, {"secondary_y": True}]])

# add Not Reg Customers
fig3.add_trace(
 go.Scatter(
  x= df_w.datetime,
  y= df_w.loc[df_w.registered_customer == False]["reg_customer"],
  name= "Unregistered Customers",
  legendgroup= "group1",
  legendgrouptitle_text= "<b>Customer Type:</b>",
  mode= "markers",
  marker= dict(color= px.colors.sequential.Brwnyl[3]),
  ),
  row= 1,
  col= 1
)

# add Reg Customers
fig3.add_trace(
 go.Scatter(
  x= df_w.datetime,
  y= df_w.loc[df_w.registered_customer == True]["reg_customer"],
  name= "Registered Customers",
  legendgroup= "group1",
  mode= "markers",
  marker= dict(color= px.colors.sequential.Brwnyl[5])
 ),
 row= 1,
 col= 2
)

# add monthly mean temperature
fig3.add_trace(
  go.Scatter(
    x= df_wth.datetime,
    y= df_wth.temp,
    marker= dict(color=px.colors.sequential.Brwnyl[1]),
    name= "Temperature (1)",
    legendgroup= "g2",
    legendgrouptitle_text= "<b>Mean Monthly Temperature</b>",
    visible= "legendonly"
  ),
  col= 1, 
  row= 1,
  secondary_y= True
)

fig3.add_trace(
  go.Scatter(
    x= df_wth.datetime,
    y= df_wth.temp,
    marker= dict(color= px.colors.sequential.Brwnyl[1]),
    name= "Temperature (2)",
    legendgroup= "g2",
    visible= "legendonly"
  ),
  col= 2, 
  row= 1,
  secondary_y= True
)

fig3.update_layout(legend=dict(groupclick="toggleitem"),title= dict(text= "<b>Weekly Customer in relation to Temperature</b>",  font_size= 20, xanchor= "center", yanchor= "top", y=0.9, x=0.43))
fig3.update_yaxes(title_text="<b>Total Customers</b>", secondary_y=False, col= 1, row=1)
fig3.update_yaxes(title_text="<b>Temperature °C</b>" , secondary_y=True, row=1, col= 2)
fig3.update_xaxes(title_text="<b>Datetime</b>")
fig3.show()

### the number of our unregistered customer correlate heavily with the temperatur

# Linear Model

In [57]:
#Linear model for registered customer
df_copy = df[df["registered_customer"] == True].copy()
df_copy = pd.get_dummies(df_copy)
df_2 = df_copy.groupby(["registered_customer",pd.Grouper(key="datetime",freq="D")]).agg(Anzahl_pro_Tag=("registered_customer","count"),temp=("temp","mean"),humidity=("humidity","mean"),windspeed=("windspeed","mean"),workingday=("workingday","mean"),holiday=("holiday","mean"),Anzahl_registered=("registered_customer","sum"),clear_few_clouds=("weather_clear, few clouds","mean"),cloudy_mist=("weather_cloudy, mist","mean"),heacy_rain_thunderstorm_snow_icepallets=("weather_heacy rain or thunderstorm or snow or ice pallets","mean"),ligth_snow_rain_thunderstorm=("weather_light snow or rain or thunderstorm","mean")).reset_index()
df_2["Anteil_registered"] = df_2["Anzahl_registered"]/df_2["Anzahl_pro_Tag"]
df_2["weekend"] = df_2["datetime"].dt.weekday > 4
df_2['date_delta'] = (df_2['datetime'] - df_2['datetime'].min())  
df_2["date_delta"] = df_2.date_delta.dt.days
df_2["sin"] = df_2.temp * np.sin(df_2.date_delta)
#df_2["temp"] = df_2.temp * df_2.temp
#df_2["temp"] = df_2.date_delta * np.log(df_2.temp)
df_2.dropna(inplace=True)
#trainings-Test-Split
features = ["sin","date_delta", 'temp', 'humidity','windspeed',"workingday","weekend","clear_few_clouds","heacy_rain_thunderstorm_snow_icepallets","ligth_snow_rain_thunderstorm"] # Liste von Features
X = df_2[features]
y = df_2.Anzahl_pro_Tag

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# other linear models that give worse predictions
# model_lr = linear_model.LinearRegression()
# model_lr.fit(X_train, y_train)
# y_train_pred = model_lr.predict(X_train)
# y_test_pred = model_lr.predict(X_test)

# reg_ridg = linear_model.Ridge(alpha=9)
# reg_ridg.fit(X_train, y_train)
# y_train_pred = reg_ridg.predict(X_train)
# y_test_pred = reg_ridg.predict(X_test)


reg = linear_model.Lasso(alpha=9)
reg.fit(X_train, y_train)
y_train_pred = reg.predict(X_train)
y_test_pred = reg.predict(X_test)

In [58]:
#print(X.columns.values)
#print('Coefficients: \n', reg.coef_)
#print('Intercept: \n', reg.intercept_)
print('On Training set:')
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_test_pred))
#print("Variance of target variable: %.2f" % np.var(y_test))
print('r2_score: %.2f' % r2_score(y_test, y_test_pred))


On Training set:
Mean squared error: 1364548.61
r2_score: 0.66


### we get a r2 score of 0.66, although with a high Mean squared error of 1364548.61 on registered customer

In [59]:
#Linear model for unregistered customer
df_copy = df[df["registered_customer"] == False].copy()
df_copy = pd.get_dummies(df_copy)
df_2 = df_copy.groupby(["registered_customer",pd.Grouper(key="datetime",freq="D")]).agg(Anzahl_pro_Tag=("registered_customer","count"),temp=("temp","mean"),humidity=("humidity","mean"),windspeed=("windspeed","mean"),workingday=("workingday","mean"),holiday=("holiday","mean"),Anzahl_registered=("registered_customer","sum"),clear_few_clouds=("weather_clear, few clouds","mean"),cloudy_mist=("weather_cloudy, mist","mean"),heacy_rain_thunderstorm_snow_icepallets=("weather_heacy rain or thunderstorm or snow or ice pallets","mean"),ligth_snow_rain_thunderstorm=("weather_light snow or rain or thunderstorm","mean")).reset_index()
df_2["Anteil_registered"] = df_2["Anzahl_registered"]/df_2["Anzahl_pro_Tag"]
df_2["weekend"] = df_2["datetime"].dt.weekday > 4
df_2['date_delta'] = (df_2['datetime'] - df_2['datetime'].min())  
df_2["date_delta"] = df_2.date_delta.dt.days
df_2["sin"] = df_2.temp * np.sin(df_2.date_delta)
#df_2["temp"] = df_2.temp * df_2.temp
#df_2["temp"] = df_2.date_delta * np.log(df_2.temp)
df_2.dropna(inplace=True)
#trainings-Test-Split
features = ["date_delta", 'temp', 'humidity','windspeed',"workingday","ligth_snow_rain_thunderstorm"] # Liste von Features
X = df_2[features]
y = df_2.Anzahl_pro_Tag

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# other linear models that give worse predictions
# model_lr = linear_model.LinearRegression()
# model_lr.fit(X_train, y_train)
# y_train_pred = model_lr.predict(X_train)
# y_test_pred = model_lr.predict(X_test)

# reg_ridg = linear_model.Ridge(alpha=9)
# reg_ridg.fit(X_train, y_train)
# y_train_pred = reg_ridg.predict(X_train)
# y_test_pred = reg_ridg.predict(X_test)

reg = linear_model.Lasso(alpha=9)
reg.fit(X_train, y_train)
y_train_pred = reg.predict(X_train)
y_test_pred = reg.predict(X_test)

In [60]:
#print(X.columns.values)
#print('Coefficients: \n', reg.coef_)
#print('Intercept: \n', reg.intercept_)
print('On Training set:')
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_test_pred))
#print("Variance of target variable: %.2f" % np.var(y_test))
print('r2_score: %.2f' % r2_score(y_test, y_test_pred))


On Training set:
Mean squared error: 219841.26
r2_score: 0.65


### we get a r2 score of 0.65, although with a high Mean squared error of 219841.626 on unregistered customer

# Head of Department

### Monthly Costumer in comparison

In [61]:
df_copy = df.copy()
df_box = df_copy.groupby(pd.Grouper(key="datetime",freq="D")).agg({"registered_customer": "count"}).reset_index()
df_box["month"] = df_box.datetime.dt.month
df_box["year"] = df_box.datetime.dt.year
df_box_2020 = df_box.loc[df_box.year == 2020]
df_box_2021 = df_box.loc[df_box.year == 2021]
df_box_2022 = df_box.loc[df_box.year == 2022]

In [62]:
# create Subplots (2 rows and 1 column) 
fig1 = make_subplots(rows= 2, cols=1, vertical_spacing= 0.1, subplot_titles= ("<b>2020</b>", "<b>2021</b>"))  

# add boxplots in row 1 for months in the Dataframe 
for month in df_box_2020.month.unique():

  if df_box_2020.registered_customer.median() < df_box_2020[df_box_2020.month == month].registered_customer.median():
    color = px.colors.sequential.Brwnyl[3]

  elif df_box_2020.registered_customer.median() > df_box_2020[df_box_2020.month == month].registered_customer.median():
    color = px.colors.sequential.Brwnyl[0]


  fig1.add_trace(
    go.Box(
      name= f"{month}",
      legendgroup= "group1",
      legendgrouptitle_text= "2020",
      x= df_box_2020.loc[df_box_2020.month == month].month,
      y= df_box_2020.loc[df_box_2020.month == month].registered_customer,
      boxpoints= "all",
      line= dict(width= 1),
      marker= dict(size=2, color= "rgb(0,0,0)"),
      jitter= 0.5,
      whiskerwidth=0.2,
      fillcolor = color,
      hoveron = "boxes+points",
      ),
      col= 1,
      row= 1
  )

# add Median 2021 in row 1
fig1.add_hline(
 y= df_box_2021.registered_customer.median(),
 opacity= 0.5,
 line_dash= "dot",
 line_color= "black",
 layer= "below",
 annotation_text= "Median 2021: "f'{df_box_2021.registered_customer.median()}',
 annotation_position= "top left",
 annotation_font_color= "black",
 annotation_opacity = 0.5,
 col= 1,
 row= 1
)

# add Median 2020 in row 1
fig1.add_hline(
 y= df_box_2020.registered_customer.median(),
 opacity= 0.5,
 line_dash= "dot",
 line_color= "black",
 layer= "below",
 annotation_text= "Median 2020: "f'{df_box_2020.registered_customer.median()}',
 annotation_position= "top left",
 annotation_font_color= "black",
 annotation_opacity = 0.5,
 col= 1,
 row= 1
)

# add boxplots in row 2 for months in Dataframe
for month in df_box_2021.month.unique():

  if df_box_2021.registered_customer.median() < df_box_2021[df_box_2021.month == month].registered_customer.median():
    color = px.colors.sequential.Brwnyl[5]

  elif df_box_2021.registered_customer.median() > df_box_2021[df_box_2021.month == month].registered_customer.median():
    color = px.colors.sequential.Brwnyl[0]

    if df_box_2020.registered_customer.median() < df_box_2021[df_box_2021.month == month].registered_customer.median():
      color = px.colors.sequential.Brwnyl[3]

  fig1.add_trace(
    go.Box(
      name= f"{month}",
      legendgroup= "group2",
      legendgrouptitle_text= "2021",
      x= df_box_2021.loc[df_box_2021.month == month].month,
      y= df_box_2021.loc[df_box_2021.month == month].registered_customer,
      boxpoints= "all",
      line= dict(width= 1),
      marker= dict(size=2, color= "rgb(0,0,0)"),
      jitter= 0.5,
      whiskerwidth=0.2,
      fillcolor = color,
      hoveron = "boxes+points"
      ),
      col= 1,
      row= 2
  )

# add Median 2021 in row 2
fig1.add_hline(
 y= df_box_2021.registered_customer.median(),
 opacity= 0.5,
 line_dash= "dot",
 line_color= "black",
 layer= "below",
 annotation_text= "Median 2021: "f'{df_box_2021.registered_customer.median()}',
 annotation_position= "top left",
 annotation_font_color= "black",
 annotation_opacity = 0.5,
 col= 1,
 row= 2
)

# add Median 2020 in row 2
fig1.add_hline(
 y= df_box_2020.registered_customer.median(),
 opacity= 0.5,
 line_dash= "dot",
 line_color= "black",
 layer= "below",
 annotation_text= "Median 2020: "f'{df_box_2020.registered_customer.median()}',
 annotation_position= "top left",
 annotation_font_color= "black",
 annotation_opacity = 0.5,
 col= 1,
 row= 2
)

# update layout 
fig1.update_layout(title= dict(text= "<b>Boxplot of total monthly Customers 2020 and 2021</b>", font_size= 20),
                   height= 770,
                   legend_title_text= "<b>Year</b>",
                   legend_title_font_size= 16,
                   legend= dict(groupclick= "toggleitem"))

# update axes
fig1.update_yaxes(title_text="<b>Total Customers</b>", title_font= dict(size= 14), showgrid= False)
fig1.update_xaxes(title_text="<b>Month</b>", title_font= dict(size= 14), col=1, row= 2)
fig1.show()

### -> the highest scooter demands happen between month 4 to 10 and the lowest between the months 12 and 2

# Monthly Spike at the Beginning and at the End

In [63]:
df_copy = df.copy()
df_2 = df_copy.groupby(pd.Grouper(key="datetime",freq="D")).agg({"registered_customer": "count"}).reset_index()
# extra dataframe to show specific time frame
df_5 = df_2[df_2.datetime.dt.date.between(datetime.date(2021,3,15),datetime.date(2021,4,15))]
df_2["spikes"] = (df_2.datetime.dt.day < 5) | (df_2.datetime.dt.day > 27)

In [64]:
# make subplots with 2 rows and 1 column
fig = make_subplots(rows= 2, cols=1, subplot_titles= ("<b>Total Customers per Day</b>", "<b>April Spike</b>")) 

# add line plot of registered_customer over time in row 1
fig.add_trace(
    go.Line(x=df_2.datetime,y=df_2.registered_customer, marker= dict(color= px.colors.sequential.Brwnyl[4]), name= "Total Customers"),
      col= 1,
      row= 1)

# add a vertical rectangle in row 1 
fig.add_vrect(x0=f"{2021}-{3}-28", 
        x1=f"{2021}-{4}-3", 
        col=1,
        annotation_text= "", 
        fillcolor= px.colors.sequential.Brwnyl[1], 
        opacity=0.3, 
        line_width=0,
        annotation_font_size = 13) 

# add the same line plot in row 2 but over a shorter time intervall
fig.add_trace(
    go.Line(x=df_5.datetime.dt.date,y=df_5.registered_customer, marker= dict(color= px.colors.sequential.Brwnyl[6]), name= "April"),
      col= 1,
      row= 2)

# add a rectangle in row 2
fig.add_vrect(x0=f"{2021}-{3}-28", 
        x1=f"{2021}-{4}-3", 
        col=1,
        annotation_text= "", 
        fillcolor= px.colors.sequential.Brwnyl[2], 
        opacity=0.3, 
        line_width=0,
        annotation_font_size = 13) 

# update the plot layout
fig.update_layout(title= dict(text= "<b>Spikes on Total Costumers at Month Begin and End</b>", xanchor= "center", yanchor= "top", y=0.98, x=0.46),
                  height= 600, legend=dict(groupclick="toggleitem"), font= dict(size= 14))

fig.update_yaxes(title_text="<b>Total Customers</b>", title_font= dict(size= 14), showgrid= False)
fig.update_xaxes(title_text="<b>Day</b>", title_font= dict(size= 14), col=1, row= 2)

fig.show()

C:\Users\paulh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




### these spikes always happen at month begin and end

### our explanation is that at month end and begin people are most willing to spend their monthly allowance

In [65]:
# make two columns with sahre x and y axes
fig = make_subplots(cols= 2, rows= 1, shared_xaxes= True, shared_yaxes= True, horizontal_spacing= 0)

# add the first boxplot with spikes == True in col 1
fig.add_trace(
 go.Box(
  x= df_2[df_2.spikes == True].spikes,
  y= df_2[df_2.spikes == True].registered_customer,
  boxpoints= "all",
  jitter= 0.5,
  whiskerwidth= 0.2,
  hoveron= "boxes+points",
  marker= dict(size=3, color= px.colors.sequential.Brwnyl[2]),
  line= dict(width=1.75),
  fillcolor= px.colors.sequential.Brwnyl[1],
  name= "True",
  legendgroup= "2",
  legendgrouptitle_text= "Spikes"
 ),
 col= 1,
 row= 1
)

# add the second boxplot with spike == False in col 2
fig.add_trace(
 go.Box(
  x= df_2[df_2.spikes == False].spikes,
  y= df_2[df_2.spikes == False].registered_customer,
  boxpoints= "all",
  jitter= 0.5,
  whiskerwidth= 0.2,
  hoveron= "boxes+points",
  marker= dict(size=3, color= px.colors.sequential.Brwnyl[6]),
  line= dict(width=1.75),
  fillcolor= px.colors.sequential.Brwnyl[4],
  name= "False",
  legendgroup= "1",
  legendgrouptitle_text= "No Spikes"
 ),
 col= 2,
 row= 1
)

# update plot layout
fig.update_layout(title= dict(text= "<b>Spikes on Total Costumers at Month Begin and End</b>", xanchor= "center", yanchor= "top", y=0.9, x=0.46), legend_title_text= "<b>Spikes</b>", legend=dict(groupclick="toggleitem"))
fig.update_yaxes(title_text="<b>Total Customers</b>", title_font= dict(size= 14), col= 1)

### demand is at month begin and end around 1500 per day higher

# share of every weekday per week

In [66]:
df_copy = df.copy()
df_3 = df.groupby(pd.Grouper(key="datetime",freq="D")).agg({"registered_customer": "count"}).reset_index()
df_3[["year","month"]] = df_3.datetime.dt.isocalendar().iloc[:,0:2] #.week funktion gibt es erst seit python 3.9
#df_3["Tag"] = pd.to_datetime(df_3["Tag"])
df_2 = df.groupby(pd.Grouper(key="datetime",freq="W")).agg({"registered_customer": "count"}).reset_index()
df_2[["year","month"]] = df_2.datetime.dt.isocalendar().iloc[:,0:2]
df_4 = pd.merge(df_3,df_2,left_on=["year","month"],right_on=["year","month"])
df_4["share_per_week"] = df_4.registered_customer_x / df_4.registered_customer_y
df_4["day"] = df_3.datetime.dt.weekday
df_4 = df_4[df_4.share_per_week < np.percentile(df_4.share_per_week,99)]


In [571]:
df_4.head()

,datetime_x,registered_customer_x,year,month,datetime_y,registered_customer_y,share_per_week,day
2,2020-01-06,1349,2020,2,2020-01-12,9408,0.143389,0
3,2020-01-07,1562,2020,2,2020-01-12,9408,0.166029,1
4,2020-01-08,1600,2020,2,2020-01-12,9408,0.170068,2
5,2020-01-09,1606,2020,2,2020-01-12,9408,0.170706,3
6,2020-01-10,1510,2020,2,2020-01-12,9408,0.160502,4


In [68]:
fig = make_subplots(cols= 1, rows= 1, shared_xaxes= True, shared_yaxes= True, horizontal_spacing= 0)

# add boxplot 
fig.add_trace(
 go.Box(
  x= df_4.day,
  y= df_4.share_per_week,
  boxpoints= "all",
  jitter= 0.5,
  whiskerwidth= 0.2,
  hoveron= "boxes+points",
  marker= dict(size=3, color= px.colors.sequential.Brwnyl[2]),
  line= dict(width=1.75),
  fillcolor= px.colors.sequential.Brwnyl[1],
  name= "True",
  legendgroup= "2",
  legendgrouptitle_text= "Spikes"
 ),
 col= 1,
 row= 1
)

# add annotation for the median
fig.add_annotation(
 x= 6.5,
 y= df_4.share_per_week.median(),
 arrowhead= 1,
 showarrow= True,
 arrowcolor= px.colors.sequential.Brwnyl[5],
 font= dict(size= 12, color= px.colors.sequential.Brwnyl[5]),
 text= f"Median: {df_4.share_per_week.median().round(4)}"
)

# add median trace
fig.add_hline(
 y= df_4.share_per_week.median(),
 opacity= 0.8,
 line_dash= "dot",
 line_color=  px.colors.sequential.Brwnyl[5],
 layer= "below"
)

# update plot layout
fig.update_layout(title= dict(text= "<b>Share of every weekday per week</b>", xanchor= "center", yanchor= "top", y=0.9, x=0.46), legend_title_text= "<b>Spikes</b>", legend=dict(groupclick="toggleitem"))
fig.update_yaxes(title_text="<b>Total Customers</b>", title_font= dict(size= 14), col= 1)
fig.update_xaxes(title_text="<b>Day</b>", title_font= dict(size= 14), col=1, row= 1)
fig.show()

### the demand over the week is relatively even distributed (only noticeable difference on sundays(bit lower))

In [69]:
df_copy = df.copy()
df_3 = df_copy.groupby(pd.Grouper(key="datetime",freq="H")).agg({"registered_customer": "count"}).reset_index()
df_3["day"] = df_3.datetime.dt.date
df_3["day"] = pd.to_datetime(df_3["day"])
df_2 = df.groupby(pd.Grouper(key="datetime",freq="D")).agg({"registered_customer": "count","workingday":"mean","holiday":"mean", "temp":"mean"}).reset_index()
df_44 = pd.merge(df_3,df_2,left_on="day",right_on="datetime")
df_44["share_per_day"] = df_44.registered_customer_x / df_44.registered_customer_y
df_44["hour"] = df_3.datetime.dt.hour
df_44["weekend"] = df_44["datetime_y"].dt.weekday > 4

In [70]:
# make a plot with 2 col and 1 row and shared x and y axes
fig = make_subplots(cols= 2, rows= 1, shared_xaxes= True, shared_yaxes= True, horizontal_spacing= 0.01)

# add boxplot with weekend == True
fig.add_trace(
 go.Box(
  x= df_44[df_44.weekend == True].hour,
  y= df_44[df_44.weekend == True].share_per_day,
  boxpoints= False,
  jitter= 0.5,
  whiskerwidth= 0.2,
  hoveron= "boxes+points",
  marker= dict(size=3, color= px.colors.sequential.Brwnyl[3]),
  line= dict(width=1.75),
  fillcolor= px.colors.sequential.Brwnyl[2],
  name= "True"
 ),
 col= 1,
 row= 1
)

# add boxplot with weekend == False
fig.add_trace(
 go.Box(
   x= df_44[df_44.weekend == False].hour,
  y= df_44[df_44.weekend == False].share_per_day,
  boxpoints= False,
  jitter= 0.5,
  whiskerwidth= 0.2,
  hoveron= "boxes+points",
  marker= dict(size=3, color= px.colors.sequential.Brwnyl[6]),
  line= dict(width=1.75),
  fillcolor= px.colors.sequential.Brwnyl[4],
  name = "False"
 ),
 col= 2,
 row= 1
)

# add highlighting rectangels
fig.add_vrect(x0= 7.5, 
              x1=8.5, 
              annotation_text= "businessday spike 1", 
              annotation_position="top left",
              fillcolor=px.colors.sequential.Brwnyl[2], 
              opacity=0.3, 
              line_width=0,
              annotation_font_color = px.colors.sequential.Brwnyl[6],
              annotation_font_size = 9,
              col= 2,
              row= 1)

fig.add_vrect(x0= 16.5, 
              x1= 18.5, 
              annotation_text= "businessday spike 2", 
              annotation_position="top left",
              fillcolor=px.colors.sequential.Brwnyl[2], 
              opacity=0.3, 
              line_width=0,
              annotation_font_color = px.colors.sequential.Brwnyl[6],
              annotation_font_size = 9,
              col= 2,
              row= 1)

# update plot layout
fig.update_layout(title= dict(text= "<b>Spikes on Total Costumers at Month Begin and End</b>", xanchor= "center", yanchor= "top", y=0.9, x=0.46), legend_title_text= "<b>Weekend</b>", legend=dict(groupclick="toggleitem"))
fig.update_yaxes(title_text="<b>Total Customers</b>", title_font= dict(size= 14), col= 1)
fig.update_xaxes(title_text="<b>Hour</b>", title_font= dict(size= 14), col=1, row= 1)
fig.update_xaxes(title_text="<b>Hour</b>", title_font= dict(size= 14), col=2, row= 1)

### -> the scooters need to be differently distributed on businessdays compared to weekends